It is a pain to have to keep dumping compute and garbage text files just to regenerate J each time.
The point of this notebook is to carefully review the process of J calculation,
so that we can perform it just once and save to the result to a shared numpy zip file.

In [1]:
%matplotlib notebook
import flux

Missing flux value for: GLEAM J202818-254302 at frequency: 204 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 212 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 220 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 227 MHz.
Missing spectral index for: GLEAM J202818-254302


//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:18: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.pos1_Elm_samples = h5f["pos1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:19: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.neg1_Elm_samples = h5f["neg1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:20: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.freq_nodes = h5f["frequencies"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:21: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.L_model = h5f["spatial_bandlimit"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:22: H5pyDeprec

delta_nu_in is 1000000.0


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import healpy as hp

In [4]:
list_az = []
list_alt = []
for source in flux.catalog.obj_catalog:
    ra = np.radians(source.ra_angle)
    dec = np.radians(source.dec_angle)
    az, alt = flux.rot.eq_to_topo(
        ra, dec,
        np.radians(flux.rot.hera_lat),
        flux.rot.get_lst(), radians=True
    )
    list_az.append(az)
    list_alt.append(alt)

In [5]:
"""
I am commenting this part out because we want to be able to
index into J as though we were indexing into obj_catalog.
list_az.sort()
list_alt.sort()
"""

# reformat into arrays so that the spline function can handle the single angles
list_az = np.array(list_az)
list_alt = np.array(list_alt)
print(list_az)
print(list_alt)

[-2.21399339  1.92973374  1.81173758 ... -1.76924241  1.75534913
 -1.91706073]
[0.06523562 1.21296969 0.4337551  ... 0.72991699 0.46504434 0.89746837]


In [6]:
#nside = 16, 32, 64, 128...
nside = 128

# healpix does the interpolation for you?
theta, phi = hp.pix2ang(nside, np.arange(12 * nside * nside))
az = phi
alt = np.pi / 2 - theta
J = flux.stokes.spline_beam_func(150e6, alt, az)

In [8]:
print(az[3])
print(list_az[3])
print(alt.dtype)
J.shape

5.497787143782138
-2.8884764391135818
float64


(196608, 2, 2)

In [9]:
print(az)
print(alt)
list_alt.dtype

[0.78539816 2.35619449 3.92699082 ... 2.35619449 3.92699082 5.49778714]
[ 1.56441744  1.56441744  1.56441744 ... -1.56441744 -1.56441744
 -1.56441744]


dtype('float64')

In [10]:
J = flux.stokes.spline_beam_func(150e6, list_alt, list_az)
J.shape

(3872, 2, 2)

In [11]:
empty_blocks = 0
for block in J:
    if block[0][0] == 0 and \
        block[1][0] == 0 and \
        block[0][1] == 0 and \
        block[1][1] == 0:
        empty_blocks += 1
empty_blocks

1484

In [12]:
""" This is a major problem, is it not? """
percent_empty = 100 * empty_blocks / len(J)
percent_empty

38.32644628099174

In [15]:
# Hard-coding a right-ascension,
# this should be a maximum response...

# the cold patch represents degrees 0 through 120

ra_max = np.radians(59.984791666666666)
dec_max = np.radians(-23.4345)

J_max = flux.stokes.J_matrix(ra, dec, lst=np.pi / 3)
print(J_max)

[[[-0.01169484+0.00335301j -0.05251387-0.01053338j]
  [-0.0521776 +0.0101595j   0.01215581+0.00316631j]]]


In [20]:
dec_decf = np.radians(-30.72)

ra_decf = 0

while ra_decf <= 120:
    J_decf = flux.stokes.J_matrix(
        np.radians(ra_decf), dec_decf, lst=np.pi / 3
    )
    print("\nDeclination:", np.around(dec_decf, 2),
          "\nRight acension:", np.around(ra_decf, 2),
          "\nResultant J matrix:\n", J_decf, "\n")
    ra_decf += 10


Declination: -0.54 
Right acension: 0 
Resultant J matrix:
 [[[-0.01394893+0.00664088j  0.03292649+0.00266418j]
  [ 0.01151518-0.03048613j  0.01611135-0.01044008j]]] 


Declination: -0.54 
Right acension: 10 
Resultant J matrix:
 [[[-0.00941275+0.00865026j  0.02039517-0.0150796j ]
  [ 0.01680992-0.02457134j  0.00893044-0.01070789j]]] 


Declination: -0.54 
Right acension: 20 
Resultant J matrix:
 [[[ 0.01164934+0.00730785j -0.02650823-0.02985053j]
  [-0.03844987-0.01168298j -0.01436482-0.00873063j]]] 


Declination: -0.54 
Right acension: 30 
Resultant J matrix:
 [[[ 0.00480075+0.00541987j -0.01722691-0.02081558j]
  [-0.02089781-0.02192923j -0.00578392-0.00637318j]]] 


Declination: -0.54 
Right acension: 40 
Resultant J matrix:
 [[[ 0.01085977+0.00071735j -0.05349354-0.01452555j]
  [-0.06220647+0.01857443j -0.01114586+0.00174292j]]] 


Declination: -0.54 
Right acension: 50 
Resultant J matrix:
 [[[-0.02113884+0.01928468j  0.27577173-0.2352934j ]
  [ 0.19527432-0.16368834j  0.0219845